In [1]:
import pandas as pd
from pymongo import MongoClient
import numpy as np

#import the data_tools needed
import sys
sys.path.insert(0, "../../data_tools/ressources/")
from mongo_tools import get_mongo_database,mongo_coll_to_dicts, mongo_to_dataframe, dataframe_to_mongo

#import the project specific data params
sys.path.insert(0, "../")
from data_params import DATABASE

#make available the db of the project and display the available collections
db = get_mongo_database(DATABASE)
db.collection_names()

[u'wb_demo_eco',
 u'full_data',
 u'results_gb_2016',
 u'grant_information',
 u'winners_1896_2008',
 u'winners_2012',
 u'results_france_2016']

In [31]:
#choose your collection and tranform it in a pd dataframe
df = mongo_to_dataframe(DATABASE,'grant_information')

In [27]:
df.head()

,Budget,DATE,NGB Name,PAID,SPORT,Year
0,Lottery,2009-09-04,British Curling Association,"54,500.00",Curling,2009
1,Lottery,2009-09-04,British Bob Skeleton Association,"39,000.00",Skeleton,2009
2,Exchequer,2009-09-04,British Curling Association,"12,250.00",Wheelchair Curling,2009
3,Exchequer,2009-09-04,British Performance Basketball Ltd,"4,500.00",Basketball,2009
4,Exchequer,2009-09-04,Disability Snowsport UK,"8,000.00",Paralympic Skiing,2009


In [32]:
df['DATE']=pd.to_datetime(df.DATE,errors='coerce')
df['Year'] = df.DATE.map(lambda x : x.year)

In [35]:
df.dropna(subset=['Year'],inplace=True)

In [65]:
df.PAID = df.PAID.str.replace(',','')

In [66]:
df.PAID = df.PAID.astype(np.float32, raise_on_error=False, copy=False)

In [68]:
#check availability of sport attachment information ==> 81% du budget est allouable à une discipline facilement
#nous nous limitons à ces éléments
df.loc[df.SPORT.notnull(), 'PAID'].sum()/df.PAID.sum()

0.81140071

In [78]:
df_alloc = df.copy()
df_alloc = df_alloc[(df_alloc.SPORT.notnull())&(df_alloc.SPORT != 'EIS')]

In [109]:
HANDICAP = r'Wheelchair|Weelchair|Paralympic|Para|Disability|Impaired|Adaptive|Boccia|Goalball'
WINTER = r'Bobsleigh|Ski|Snow|Curling|Skeleton|Skating'

In [110]:
df_alloc.loc[df_alloc.SPORT.str.contains(HANDICAP),'category'] = 'Paralympic' 
df_alloc.loc[df_alloc.SPORT.str.contains(WINTER),'category'] = 'Winter'
df_alloc.loc[df_alloc.category.isnull(),'category'] = 'Summer' 

In [113]:
df_summer.SPORT.unique()

array([u'Basketball', u'Swimming', u'Equestrian', u'Diving', u'Hockey',
       u'Volleyball', u'Rowing', u'Athletics', u'Synchronised Swimming',
       u'Badminton', 'Gymnastics', u'Judo', u'Archery', 'Shooting',
       u'Handball', u'Boxing', 'Table Tennis', u'Taekwondo', u'Triathlon',
       u'Sailing', u'Water Polo', u'Wrestling', u'Cycling',
       u'Modern Pentathlon', u'Fencing', u'Canoeing', u'Powerlifting',
       u'Weightlifting'], dtype=object)

In [111]:
df_summer = df_alloc.copy()
df_summer = df_summer[df_summer.category=='Summer']
del df_summer['category']

In [89]:
#après avoir retraité les jeux d'hiver et paralympiques notre analyse concerne 70% du budget de uksport
#http://www.uksport.gov.uk/resources/business-plan, 350m investis tout compris on the road to Rio ==> le remarquer en commentaire
df_summer.PAID.sum()/df.PAID.sum()

0.69534254

In [103]:
RENAME = {
    "Gymnastics - Women's Artistic":'Gymnastics',
    "Gymnastics - Men's Artistic":'Gymnastics',
    'Target Shooting':'Shooting',
    'Table Tennis (GB)':'Table Tennis'
}

In [112]:
df_summer.SPORT = df_summer.SPORT.replace(RENAME)